<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
由 <a href="https://sebastianraschka.com">Sebastian Raschka</a> 撰写的 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> 一书的补充代码<br>
<br>代码仓库：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第三章：编码注意力机制

本notebook会用到以下第三方库

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.2.1+cu118


- 本章节覆盖LLMs的引擎--注意力机制

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="600px">
</p>

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="800px">
</p>

## 3.1 建模长序列的问题
- 本小结没有代码
- 在不同语种文本翻译任务中，因为不同语言的语法结构不同导致逐词翻译是不可行的

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">
</p>

- 在transformer模型出现之前，编码器-解码器RNN通常用于机器翻译任务
- 在这种设置中，编码器处理源语言的标记序列，使用隐藏状态（神经网络内部的一种中间层）来生成整个输入序列的压缩表征

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="600px">
</p>

## 3.2 使用注意力机制捕捉数据之间的依赖
- 本小结没有代码
- 通过注意力机制，网络的文本生成解码器部分能够选择性地访问所有输入标记，这意味着在生成特定输出标记时，某些输入标记比其他标记更重要

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="600px">
</p>

- 自注意力在transformer中是一种旨在增强输入表示的技术，它使序列中的每个位置都能与同一序列中的所有其他位置进行交互并确定它们的相关性

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">
</p>

## 3.3 通过自注意力关注输入的不同部分

### 3.3.1 一个不带可训练权重的简单自注意力机制
- 本节解释了一个非常简化的自注意力变体，不包含任何可训练权重
- 纯粹是为了说明目的，不是transformer中使用的注意力机制
- 下一节，3.3.2节，将扩展这个简单的注意力机制来实现真正的自注意力机制
- 假设有一个输入序列$x^{(1)}$到$x^{(T)}$
  - 输入是一段文本（例如，一个句子如"Your journey starts with one step"），已经按照第2章所述转换为标记嵌入
  - 例如，$x^{(1)}$是表示单词"Your"的d维向量，以此类推
- **目标：**为输入序列$x^{(1)}$到$x^{(T)}$中的每个输入序列元素$x^{(i)}$计算上下文向量$z^{(i)}$（其中$z$和$x$具有相同的维度）
    - 上下文向量$z^{(i)}$是输入$x^{(1)}$到$x^{(T)}$的加权和
    - 上下文向量是针对特定输入的"上下文"特定的
      - 不使用表示任意输入标记占位符$x^{(i)}$来说明，直接以第二个输入$x^{(2)}$进行举例
      - 继续使用具体示例，不用占位符$z^{(i)}$，直接使用对应的输出上下文向量$z^{(2)}$
      - 第二个上下文向量$z^{(2)}$是所有输入$x^{(1)}$到$x^{(T)}$的加权和，权重基于第二个输入元素$x^{(2)}$
      - 注意力权重是决定在计算$z^{(2)}$时每个输入元素贡献多少的权重
      - 简而言之，可以将$z^{(2)}$视为$x^{(2)}$的修改版本，它还包含了与给定任务相关的所有其他输入元素的信息

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="600px">
</p>

- （请注意，此图中的数字截断为小数点后一位，以减少视觉混乱；类似地，其他图表也可能包含截断的值）

- 按照惯例，未归一化的注意力权重被称为 **"注意力分数"**，而归一化后和为1的注意力分数则被称为 **"注意力权重"**

- 下面的代码逐步解释了上图中的过程

<br>

- 第一步：计算未归一化的注意力分数 $\omega$
- 假设使用第二个input token作为query，即 $q^{(2)} = x^{(2)}$，通过以下点乘操作计算为归一化分数
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 上面的 $\omega$ 是希腊字母"omega"，用来表示未归一化的注意力分数
    - 下标"21"在$\omega_{21}$中表示输入序列元素2被用作针对输入序列元素1的查询

- 假设有以下输入句子，已经按照第3章所述嵌入到3维向量中（这里为了说明目的使用非常小的嵌入维度，这样它可以在页面上不换行地显示）

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- （在本书中遵循常见的机器学习和深度学习惯例，其中训练样本表示为行，特征值表示为列；在上面显示的张量中，每一行代表一个词，每一列代表一个嵌入维度）

- 本节的主要目标是演示如何使用输入序列中第二个元素$x^{(2)}$作为查询来计算上下文向量$z^{(2)}$

- 图中描述了这个过程的初始步骤，即通过点积操作计算$x^{(2)}$与所有其他输入元素之间的注意力分数ω

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="600px">
</p>

- 使用输入序列元素2，$x^{(2)}$，作为示例来计算上下文向量$z^{(2)}$；在本节后面，将推广这一点来计算所有上下文向量。

- 第一步是通过计算查询$x^{(2)}$与所有其他输入标记之间的点积来计算未归一化的注意力分数

In [3]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 附注：点积本质上是将两个向量按元素相乘并对结果求和的简写

In [4]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **步骤2**：归一化未归一化的注意力分数（"omega"，$\omega$），使它们的总和为1
- 这里是一种将未归一化注意力分数归一化为总和为1的简单方法（这是一种惯例，有助于解释，并且对训练稳定性很重要）

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">
</p>

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实践中，使用softmax函数进行归一化是常见且推荐的，它更善于处理极端值，并且在训练期间具有更理想的梯度特性
- 这里是一个带缩放的softmax函数的简单实现，它也将向量元素归一化，使它们的总和为1

In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上面的简单实现可能会因为大或小的输入值而遇到数值不稳定问题，可能是溢出和下溢导致的
- 因此，在实践中，建议使用PyTorch的softmax实现，它已经为性能进行了高度优化

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **步骤3**：计算上下文向量$z^{(2)}$，方法是将嵌入的输入标记$x^{(i)}$与注意力权重相乘，然后对结果向量求和

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">
</p>

In [8]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 对所有输入tokens计算注意力权重

#### 推广到所有输入序列tokens

- 上面已计算了输入2的注意力权重和上下文向量（如下图中突出显示的行所示）
- 接下来，将这种计算推广到计算所有注意力权重和上下文向量

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">
</p>

- （请注意，此图中的数字截断为小数点后两位，以减少视觉混乱；每行中的值应加起来等于1.0或100%；类似地，其他图中的数字也被截断）

- 自注意力过程从计算注意力分数开始，然后将其归一化以得到总和为1的注意力权重
- 这些注意力权重随后通过输入的加权求和来生成上下文向量

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">
</p>

- 将之前的**步骤1**应用于所有成对元素，计算未归一化的注意力分数矩阵

In [9]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 可以通过矩阵乘法更高效地实现与上述相同的结果

In [10]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与之前的**步骤2**类似，对每一行进行归一化，使每行中的值总和为1

In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每行中的值确实总和为1

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0


All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用之前的**步骤3**来计算所有上下文向量

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 作为合理性检查，之前计算的上下文向量$z^{(2)} = [0.4419, 0.6515, 0.5683]$可以在上面的第2行找到

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 带可训练参数的自注意力实现
- 一个概念框架，说明本节开发的自注意力机制如何融入本书和本章的整体叙述和结构

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="600px">
</p>

### 3.4.1 一步一步计算注意力权重
- 本小节实现原始transformer架构中的自注意力机制，如GPT模型和大量其他流行的LLMs中均使用
- 自注意力机制也称为缩放点积注意力
- 整体思想与之前类似
  - 希望计算特定输入元素相关的输入向量的加权求和，以得到上下文向量
  - 为此，需要注意力权重
- 正如将看到的，与之前介绍的基本注意力机制相比，只有轻微的差异
  - 最显著的区别是引入了在模型训练期间更新的权重矩阵
  - 这些可训练的权重矩阵至关重要，使模型（特别是模型内部的注意力模块）能够学习产生"好的"上下文向量

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">
</p>

- 逐步实现自注意力机制，将首先介绍三个训练权重矩阵$W_q$、$W_k$和$W_v$
- 这三个矩阵用于通过矩阵乘法将嵌入的输入标记$x^{(i)}$投影到查询、键和值向量：
  - Query vector: $q^{(i)} = W_q x^{(i)}$
  - Key vector: $k^{(i)} = W_k x^{(i)}$
  - Value vector: $v^{(i)} = W_v x^{(i)}$

- 输入$x$和查询向量$q$的嵌入维度可以相同或不同，这取决于模型的设计和具体实现
- 在GPT模型中，输入和输出维度通常相同，但为了说明目的，为了更好地跟踪计算过程，在这里选择不同的输入和输出维度

In [ ]:
x_2 = inputs[1]  # 第二个输入元素
d_in = inputs.shape[1]  # 输入嵌入维度，d=3
d_out = 2  # 输出嵌入维度，d=2

- 下面，初始化三个权重矩阵；注意，设置requires_grad=False以减少输出中的混乱，这只是为了说明目的，但如果要将权重矩阵用于模型训练，会设置requires_grad=True以在模型训练期间更新这些矩阵

In [16]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 接下来计算查询、键和值向量

In [17]:
query_2 = x_2 @ W_query  # 2表示与第二个输入元素对应
key_2   = x_2 @ W_key
value_2 = x_2 @ W_value

print("Query:", query_2)
print("Key:", key_2)
print("Value:", value_2)

Query: tensor([0.4306, 1.4551])
Key: tensor([0.4433, 1.1419])
Value: tensor([0.3951, 1.0037])


- 如以下所示，成功将6个输入tokens从3D嵌入空间转换到了2D嵌入空间

In [18]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 接下来如上述的**步骤2**一致，通过查询/query向量和键/key向量之间进行点积操作计算未归一化的注意力分数

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">
</p>

In [19]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 因为输入有6个tokens，所有对于给定的查询向量应该有六个注意力分数

In [20]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">
</p>

- 接下来如上述的**步骤3**一致，使用softmax计算归一化后的注意力权重
- 与之前的区别在于，现在通过将注意力分数除以嵌入维度的平方根$\sqrt{d_k}$（即d_k**0.5）来缩放它们：

In [21]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">
</p>

- 最后，如上述的**步骤4**一致，同构加权求和计算上下文向量

In [22]:
context_vec_2 = attn_weights_2 @ values  # @ 表示矩阵乘法
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 实现一个紧凑的SelfAttention类
- 将上述所有步骤组合成一个紧凑的SelfAttention类，具体如下

In [23]:
import torch.nn as nn


class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega，未归一化的注意力分数
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec


torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">
</p>

- 可以使用PyTorch的Linear层来简化上面的实现，如果禁用偏置单元，这等同于矩阵乘法
- 使用nn.Linear而不是手动的nn.Parameter(torch.rand(...))的另一个大优势是，nn.Linear有一个首选的权重初始化方案，这会导致更稳定的模型训练

In [24]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec


torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 请注意，SelfAttention_v1和SelfAttention_v2给出不同的输出，因为它们对权重矩阵使用不同的初始权重

## 3.5 因果注意力解析
- 在因果注意力中，对角线以上的注意力权重被掩蔽，确保对于任何给定的输入，LLM在使用注意力权重计算上下文向量时无法利用未来的标记。
- 与实际场景对应，真实场景中预测下一个token任务是看不到后续的tokens的，当前token只与前面的tokens和其自己计算注意力，这就是因果注意力

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="600px">
</p>

### 3.5.1 构建因果注意力掩码
- 本节将之前的自注意力机制转换为因果自注意力机制
- 因果自注意力确保模型对序列中某个位置的预测仅依赖于先前位置的已知输出，而不依赖于未来位置
- 简单来说，这确保每个下一个词的预测应该只依赖于前面的词
- 为实现这一点，对于每个给定的标记，掩蔽掉未来的标记（在输入文本中当前标记之后出现的标记）

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">
</p>

- 为了说明和实现因果自注意力，基于前一节的注意力分数和权重讲解

In [25]:
# 重用前一节SelfAttention_v2对象的查询和键权重矩阵，以方便使用
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 最简单的掩蔽未来注意力权重的方法是通过PyTorch的tril函数创建一个掩码，将主对角线以下的元素（包括对角线本身）设置为1，主对角线以上的元素设置为0

In [26]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，可以将注意力权重与这个掩码相乘，将对角线以上的注意力分数置零

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 然而，如果掩码在softmax之后应用，如上所示，它会破坏softmax创建的概率分布
- Softmax确保所有输出值的总和为1
- 在softmax之后进行掩码处理将需要重新归一化输出，使其再次总和为1，这会使过程复杂化并可能导致意外效果

- 为了保证每行的注意力权重总和为1，可以按以下方式进行注意力权重正则化

In [28]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 虽然现在在技术上已经完成了因果注意力机制的编码，但还有更高效的方法实现与上述相同结果
- 可以在未归一化的注意力分数进入softmax函数之前，用负无穷大掩蔽对角线以上的分数，而不是将对角线以上的注意力权重置零并重新归一化结果

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">
</p>

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
# 将上三角元素全部设置为-inf，在softmax计算时，e的-inf次方为0，所以这些元素对最终的注意力权重没有贡献
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 如下所示，现在每行中的注意力权重正确地再次总和为1

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 使用dropout掩蔽额外的注意力权重
- 此外，还应用dropout来减少训练期间的过拟合
  - 例如，在计算注意力权重之后
  - 或者在将注意力权重与值向量相乘之后
- 这里，将在计算注意力权重之后应用dropout掩码，因为这更常见
- 此外，在这个特定示例中，使用50%的dropout率，这意味着随机掩蔽一半的注意力权重。（后续训练GPT模型时，将使用较低的dropout率，如0.1或0.2）

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="600px">
</p>

- 如果应用0.5（50%）的dropout率，未被丢弃的值将相应地按1/0.5 = 2的因子进行缩放；目的是为了**保持训练和测试阶段神经网络的输出期望一致**，因为预测时不会应用dropout
- 缩放率是通过公式1 / (1 - dropout_rate)计算的


In [36]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [37]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


- 请注意，根据操作系统，生成的dropout输出可能看起来不同；可以在[PyTorch问题跟踪器上的这里](https://github.com/pytorch/pytorch/issues/121595)阅读有关此不一致性的更多信息

### 3.5.3 实现一个紧凑的因果自注意力类
- 现在，准备实现一个可工作的自注意力实现，包括因果和dropout掩码
- 还有一件事是要实现处理包含多个输入的批次的代码，以便CausalAttention类支持由在第2章实现的加载数据器产生的批次输出
- 为简单起见，为了模拟这种批次输入，复制上述inputs示例

In [38]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [39]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length),
                                                diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method. 
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

In [40]:
torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 注意dropout只在训练过程中应用，推理时不会使用

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="600px">
</p>

## 3.6 从单头注意力扩展为多头注意力

### 3.6.1 堆叠多个单头注意力层
- 下面是之前实现的自注意力的总结（为简单起见，未显示因果和dropout掩码）
- 这也被称为单头注意力

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="600px">
</p>

- 只需堆叠多个单头注意力模块，即可获得多头注意力模块

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="600px">
</p>

- 多头注意力背后的主要思想是使用不同的、学习到的线性投影多次（并行）运行注意力机制。这允许模型共同关注来自不同位置的不同表示子空间的信息

In [41]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [42]:
torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上面的实现中，嵌入维度是4，因为将d_out=2作为键、查询和值向量以及上下文向量的嵌入维度。由于有2个注意力头，所以输出嵌入维度为2*2=4

### 3.6.2 通过权重分割实现多头注意力
- 虽然上面是多头注意力的直观且功能完整的实现（包装了早先的单头注意力CausalAttention实现），但可以编写一个名为MultiHeadAttention的独立类来实现相同的功能
- 对于这个独立的MultiHeadAttention类，不直接堆叠单个注意力头
- 相反，创建单个W_query、W_key和W_value权重矩阵，然后将它们分割成每个注意力头的单独矩阵

In [43]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forwar

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [44]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，上面本质上是MultiHeadAttentionWrapper的重写版本，更加高效
- 由于随机权重初始化不同，结果输出看起来有点不同，但两者都是功能完整的实现，可以在接下来章节中实现的GPT类中使用

---

**关于输出尺寸的说明**

- 在上述`MultiHeadAttention`中，设置`d_out=2`来保证其与`MultiHeadAttentionWrapper`类中设置相同
- 在`MultiHeadAttentionWrapper`中，由于拼接，返回输出头维度`d_out * num_heads` （即`2*2 = 4`）
- 然而，`MultiHeadAttention`类（为了更用户友好）允许通过`d_out`直接控制输出头维度；这意味着，如果设置`d_out = 2`，输出头维度将是2，无论有多少个头

---

- 注意，此外，在上面的MultiHeadAttention类中添加了一个线性投影层（self.out_proj）。这只是一个不改变维度的线性变换。在LLM实现中使用这样的投影层是标准惯例，但并不是严格必要的（最近的研究表明，可以在不影响建模性能的情况下移除它；请参阅本章末尾的进一步阅读部分）

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">
</p>

- 请注意，如果对上述内容的紧凑高效实现感兴趣，也可以参考PyTorch中的[`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)类
- 由于上述实现乍看之下可能有点复杂，先看看执行attn_scores = queries @ keys.transpose(2, 3)时会发生什么

In [45]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch中的矩阵乘法实现将处理4维输入张量，使矩阵乘法在最后2个维度（num_tokens, head_dim）之间进行，然后为各个头部重复
- 例如，以下分别计算每个头部矩阵乘法的更紧凑方式

In [46]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 总结和收获
- 请查看[./multihead-attention.ipynb](./multihead-attention.ipynb)代码notebook，是数据加载器（第2章）加上在本章实现的多头注意力类的简洁版本，将在接下来的章节中需要它来训练GPT模型
- 可以在[./exercise-solutions.ipynb](./exercise-solutions.ipynb)中找到练习解答